In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()
    
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [3]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [12]:
print("Text to Sequences: ", tokenizer.texts_to_sequences(["First"]))

print("Sequences to Text: ", tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]]))

Text to Sequences:  [[20, 6, 9, 8, 3]]
Sequences to Text:  ['f i r s t']


In [6]:
max_id = len(tokenizer.word_index) # number of distinct characters
print("number of distinct characters: ", max_id)
dataset_size = tokenizer.document_count # total number of characters
print("total number of characters: ", dataset_size)

number of distinct characters:  39
total number of characters:  1115394


In [7]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [8]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [9]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [10]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [11]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [12]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [13]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [30]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)
model.save('char_rnn.h5')

Epoch 1/10
31368/31368 [==============================] - 3300s 105ms/step - loss: 1.6206
Epoch 2/10
31368/31368 [==============================] - 3278s 105ms/step - loss: 1.5341 - lo
Epoch 3/10
31368/31368 [==============================] - 3221s 103ms/step - loss: 1.5124
Epoch 4/10
31368/31368 [==============================] - 3253s 104ms/step - loss: 1.5002
Epoch 5/10
31368/31368 [==============================] - 3255s 104ms/step - loss: 1.4926
Epoch 6/10
31368/31368 [==============================] - 3277s 104ms/step - loss: 1.4877
Epoch 7/10
31368/31368 [==============================] - 3285s 105ms/step - loss: 1.4834
Epoch 8/10
31368/31368 [==============================] - 3317s 106ms/step - loss: 1.4805
Epoch 9/10
31368/31368 [==============================] - 3350s 107ms/step - loss: 1.4777
Epoch 10/10
31368/31368 [==============================] - 3240s 103ms/step - loss: 1.4759


In [14]:
from keras.models import load_model
 
# load model
model = load_model('char_rnn.h5')

In [15]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

X_new = preprocess(["How are yo"])
#Y_pred = model.predict_classes(X_new)
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'u'

In [16]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [17]:
print(complete_text("t", temperature=0.2))

the devilish state,
and that i can be so a suitors 


In [18]:
print(complete_text("b", temperature=0.1))

be to be gone.

gremio:
what, hold you now, sir, i 
